In [ ]:
# Sample strategy: Buy ATM Call Option at the start of each table (day), hold till expiry
from backtesting_opt import _Data, Strategy, Backtest
import pandas as pd
import math
from datetime import datetime, date as DateObject # Added DateObject
iv = {}
class IV_Slope(Strategy):
    # Define parameters as class variables for optimization/flexibility
    iv_slope_thresholds: dict = None
    legs: dict = None
    iv: dict = None
    position_id= 0
    signal= 0
    

    def init(self):
        super().init()
        self.entry_type_dict = None
        print(legs)

    def next(self):
        super().next()
        if self.spot is None or pd.isna(self.spot):
            return
        atm = round(self.spot / 50) * 50

        for leg in self.legs.values():
            valid_tte = min(tte for tte in self.tte_to_expiry.keys() if any(lower <= tte <= upper for lower, upper in [leg["expiry_range"]]))
            leg["expiry"] = self.tte_to_expiry[valid_tte]

        for leg in self.legs.values():
            if leg["target_strike"] == "ATM":
                leg["strike"] = float(atm)
            contract = f"NIFTY{pd.Timestamp(leg['expiry']).strftime('%d%b%y').upper()}{int(leg['strike'])}{leg['type']}"
            leg["contract"] = contract
            leg["data"] = self._data.get_ticker_data(contract)

        missing_legs = [leg["contract"] for leg in self.legs.values() if leg["data"] is None]
        if missing_legs:
            print(f"IV not found for {self.time}. Spot: {self.spot} Missing legs: {missing_legs}")
            return
        
        # if (pd.Timestamp("15:29:00").time() <= pd.Timestamp(row.Index).time() <= pd.Timestamp("15:30:00").time()):
        iv_slope = math.log((self.legs["leg1"]["data"]["iv"] + self.legs["leg2"]["data"]["iv"]) / (self.legs["leg3"]["data"]["iv"] + self.legs["leg4"]["data"]["iv"]) ,10)
        # print(f"{self.spot}  {self.legs['leg1']['data']['iv']} {self.legs['leg2']['data']['iv']} {self.legs['leg3']['data']['iv']} {self.legs['leg4']['data']['iv']}")
        iv[self.time]=(iv_slope, self.spot)

        new_signal = (iv_slope > iv_slope_thresholds["upper_gamma"]) * 3 + (iv_slope_thresholds["upper_gamma"] >= iv_slope > iv_slope_thresholds["upper_buffer"]) * 2 + (iv_slope_thresholds["upper_buffer"] >= iv_slope > 0) * 1\
            + (0 >= iv_slope > iv_slope_thresholds["lower_buffer"]) * -1 + (iv_slope_thresholds["lower_buffer"] >= iv_slope > iv_slope_thresholds["lower_gamma"]) * -2 + (iv_slope_thresholds["lower_gamma"] >= iv_slope) * -3
        
        print(f"Signal: {self.signal}, new_signal: {new_signal} IV Slope: {iv_slope} Spot: {self.spot} Time: {self.time}")
        # print(f"New Signal: {new_signal} IV Slope: {iv_slope} Spot: {self.spot}")
        active_trades = self.active_trades

        if (not active_trades) and (pd.Timestamp(self.time).time() < pd.Timestamp("15:00:00").time()):
            if new_signal == -2 or new_signal == 2:         # No trade entry if buffer zone and no active position
                return
            elif new_signal == 1:
                self.entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}          # Original
                # entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}
                # continue
            elif new_signal == -1:
                self.entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}          # Original
                # entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}  
                # continue
            elif new_signal == -3 or new_signal == 3:
                self.entry_type_dict = {'weekly': 'BUY', 'monthly': None}          # Original
                # entry_type_dict = {'weekly': 'SELL', 'monthly': None}
                # continue
            
            placed_any_leg = False
            for leg_id, leg in self.legs.items():
                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                if order_fn is None:
                    continue

                order_fn(
                    strategy_id='strat1',
                    position_id=self.position_id,
                    leg_id=leg_id,
                    ticker=leg["contract"],
                    quantity=1,
                    stop_loss=None,
                    take_profit=None,
                    tag=f'{new_signal} signal entry'
                )
                placed_any_leg = True
            if placed_any_leg:
                self.position_id += 1
            
        else:
            print(f"MOMOrders: {len(self.orders)}, Active Trades: {len(self.active_trades)}, Equity: {self.equity:.2f}, closed_trades: {len(self.closed_trades)}")

            # Exit if near expiry date is reached
            near_expiry = None
            for trade in active_trades:
                expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
                near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)
            exit_reason = (
                "Near Expiry reached" if (pd.Timestamp(self.time).date() == near_expiry) else
                "Signal changed" if (self.signal != new_signal) else
                None
            )
            if exit_reason:
                for trade in active_trades:
                    contract = trade.ticker
                    print("closing position")
                    trade.close(trade.size, tag=exit_reason)
            print(f"Orders: {len(self.orders)}, Active Trades: {len(self.active_trades)}, Equity: {self.equity:.2f}, closed_trades: {len(self.closed_trades)}")


            if self.signal == new_signal:
                leg_strike = self.legs["leg2"]["strike"]
                if (self.spot*0.99) <= leg_strike <= (self.spot*1.01):
                    # Case (a)
                    pass
                else:
                    # Case (b)
                    # take new ATM Calendar
                    placed_any_leg = False
                    for leg_id, leg in self.legs.items():
                        entry_type = self.entry_type_dict.get(leg["expiry_type"])
                        order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                        if order_fn is None:
                            continue

                        order_fn(
                            strategy_id='strat1',
                            position_id=self.position_id,
                            leg_id=leg_id,
                            ticker=leg["contract"],
                            quantity=1,
                            stop_loss=None,
                            take_profit=None,
                            tag=f'Adjustment Calendar'
                        )
                        placed_any_leg = True
                    if placed_any_leg:
                        self.position_id += 1

        self.signal = new_signal



# Run the backtest pipeline
if __name__ == "__main__":
    db_path = "nifty_1min_desiquant.duckdb"
    iv_slope_thresholds = {
        "upper_gamma": 0.15,
        "upper_buffer": 0.05,
        "lower_buffer": -0.10,
        "lower_gamma": -0.3
    }

    portfolio_sl = 0.01
    portfolio_tp = 0.03
    legs = {
        'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss':None, 'take_profit':None},
        'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss':None, 'take_profit':None},
        'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss':None, 'take_profit':None},
        'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss':None, 'take_profit':None}
        }
    bt = Backtest(
        db_path=db_path,
        strategy=IV_Slope,
        cash=10000000,
        commission_per_contract=0.65,
        option_multiplier=75
    )
    processed_orders, final_positions, closed_trades, orders = bt.run(iv_slope_thresholds=iv_slope_thresholds, legs=legs)
    print("Processed Orders:", processed_orders)
    print("Final Positions:", final_positions)
    import pprint
    pprint.pprint(legs)

Initializing strategy...
{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}, 'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}, 'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}, 'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}}


Backtesting Options Strategy: 0it [00:00, ?it/s]

Processed Orders: []
Final Positions: {'Cash': 10000000}
{'leg1': {'expiry_range': [12, 20],
          'expiry_type': 'weekly',
          'stop_loss': None,
          'take_profit': None,
          'target_strike': 'ATM',
          'type': 'CE'},
 'leg2': {'expiry_range': [12, 20],
          'expiry_type': 'weekly',
          'stop_loss': None,
          'take_profit': None,
          'target_strike': 'ATM',
          'type': 'PE'},
 'leg3': {'expiry_range': [26, 34],
          'expiry_type': 'monthly',
          'stop_loss': None,
          'take_profit': None,
          'target_strike': 'ATM',
          'type': 'CE'},
 'leg4': {'expiry_range': [26, 34],
          'expiry_type': 'monthly',
          'stop_loss': None,
          'take_profit': None,
          'target_strike': 'ATM',
          'type': 'PE'}}


In [121]:
import pandas as pd
from questdb_query import pandas_query, numpy_query
from questdb_query import Endpoint
import datetime
import time

endpoint = Endpoint(host='qdb3.twocc.in', port=None, https=True, username='2Cents', password='2Cents1012cc')

def get_all_contracts(table_name):
    start_time = time.time()
    np_arrs = pandas_query(f"SELECT * FROM {table_name} ORDER BY timestamp DESC LIMIT 1", endpoint, 7)
    df = pd.DataFrame(np_arrs)
    end_time = time.time()
    print(f"Time taken: {end_time-start_time}, Downloaded {len(np_arrs)} bytes")




get_all_contracts('option_data')
for i in range(100):
    get_all_contracts('option_data')

Time taken: 0.13735055923461914, Downloaded 1 bytes
Time taken: 0.12090516090393066, Downloaded 1 bytes
Time taken: 0.21312975883483887, Downloaded 1 bytes
Time taken: 0.08232831954956055, Downloaded 1 bytes
Time taken: 0.1484060287475586, Downloaded 1 bytes
Time taken: 0.14188361167907715, Downloaded 1 bytes
Time taken: 0.12731575965881348, Downloaded 1 bytes
Time taken: 0.1269381046295166, Downloaded 1 bytes
Time taken: 0.11947274208068848, Downloaded 1 bytes
Time taken: 0.14366436004638672, Downloaded 1 bytes
Time taken: 0.1374824047088623, Downloaded 1 bytes
Time taken: 0.12248873710632324, Downloaded 1 bytes
Time taken: 0.08012914657592773, Downloaded 1 bytes
Time taken: 0.10383462905883789, Downloaded 1 bytes
Time taken: 0.12459826469421387, Downloaded 1 bytes
Time taken: 0.13818860054016113, Downloaded 1 bytes
Time taken: 0.075225830078125, Downloaded 1 bytes
Time taken: 0.14248251914978027, Downloaded 1 bytes
Time taken: 0.1276848316192627, Downloaded 1 bytes
Time taken: 0.1509

In [129]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

import aiohttp
import numpy as np
import pandas as pd

def _new_session(endpoint, timeout: int = None):
    auth = None
    if endpoint.username:
        auth = aiohttp.BasicAuth(endpoint.username, endpoint.password)
    timeout = aiohttp.ClientTimeout(total=timeout) \
        or aiohttp.ClientTimeout(total=300)
    return aiohttp.ClientSession(
        auth=auth,
        # read_bufsize=4 * 1024 * 1024,
        timeout=timeout)


def _auth_headers(endpoint: Endpoint) -> dict[str, str]:
    if endpoint.token:
        return {'Authorization': f'Bearer {endpoint.token}'}
    return None

async def main(query, session, endpoint):
    start_time = time.time()
    url = f'{endpoint.url}/exp'
    params = [('query', query)]
    
    async with session.get(url=url, params=params, headers=_auth_headers(endpoint)) as resp:
        if resp.status != 200:
            raise ValueError(f'Failed to query QuestDB: {resp.status} {resp.reason}')
        
        buf = await resp.content.read()
        end_time = time.time()
        print(f"Time taken: {end_time-start_time}, Downloaded {len(buf)} bytes")

# Run the async function
query = "SELECT * FROM option_data ORDER BY timestamp DESC LIMIT 1"
session = _new_session(endpoint)
await main(query, session, endpoint)
for i in range(100):
    asyncio.run(main(query, session, endpoint))
session.close()


Time taken: 0.13472580909729004, Downloaded 409 bytes
Time taken: 0.020598649978637695, Downloaded 409 bytes
Time taken: 0.014838218688964844, Downloaded 409 bytes
Time taken: 0.024129152297973633, Downloaded 409 bytes
Time taken: 0.015920400619506836, Downloaded 409 bytes
Time taken: 0.0164031982421875, Downloaded 409 bytes
Time taken: 0.024979591369628906, Downloaded 409 bytes
Time taken: 0.017923831939697266, Downloaded 409 bytes
Time taken: 0.02978038787841797, Downloaded 409 bytes
Time taken: 0.015700817108154297, Downloaded 409 bytes
Time taken: 0.032692670822143555, Downloaded 409 bytes
Time taken: 0.01697063446044922, Downloaded 409 bytes
Time taken: 0.0247952938079834, Downloaded 409 bytes
Time taken: 0.017642974853515625, Downloaded 409 bytes
Time taken: 0.015102624893188477, Downloaded 409 bytes
Time taken: 0.014344930648803711, Downloaded 409 bytes
Time taken: 0.01847672462463379, Downloaded 409 bytes
Time taken: 0.01636219024658203, Downloaded 409 bytes
Time taken: 0.01356

<coroutine object ClientSession.close at 0x000001C1A4EDFC40>

In [134]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

import aiohttp
import numpy as np
import pandas as pd

def _new_session(endpoint, timeout: int = None):
    auth = None
    if endpoint.username:
        auth = aiohttp.BasicAuth(endpoint.username, endpoint.password)
    timeout = aiohttp.ClientTimeout(total=timeout) \
        or aiohttp.ClientTimeout(total=300)
    return aiohttp.ClientSession(
        auth=auth,
        # read_bufsize=4 * 1024 * 1024,
        timeout=timeout)


def _auth_headers(endpoint: Endpoint) -> dict[str, str]:
    if endpoint.token:
        return {'Authorization': f'Bearer {endpoint.token}'}
    return None

async def main(query, session, endpoint):
    start_time = time.time()
    url = f'{endpoint.url}/exp'
    params = [('query', query)]
    
    async with session.get(url=url, params=params, headers=_auth_headers(endpoint)) as resp:
        if resp.status != 200:
            raise ValueError(f'Failed to query QuestDB: {resp.status} {resp.reason}')
        
        buf = await resp.content.read()
        end_time = time.time()
        print(f"Time taken: {end_time-start_time}, Downloaded {len(buf)} bytes")

# Run the async function
query = "SELECT * FROM option_data ORDER BY timestamp DESC LIMIT 1"
session = _new_session(endpoint)
# await main(query, session, endpoint)
for i in range(100):
    asyncio.run(main(query, session, endpoint))
session.close()


Time taken: 0.09870481491088867, Downloaded 409 bytes
Time taken: 0.013270378112792969, Downloaded 409 bytes
Time taken: 0.01658344268798828, Downloaded 409 bytes
Time taken: 0.018043994903564453, Downloaded 409 bytes
Time taken: 0.015815019607543945, Downloaded 409 bytes
Time taken: 0.019558191299438477, Downloaded 409 bytes
Time taken: 0.013679742813110352, Downloaded 409 bytes
Time taken: 0.015366554260253906, Downloaded 409 bytes
Time taken: 0.02446126937866211, Downloaded 409 bytes
Time taken: 0.020600557327270508, Downloaded 409 bytes
Time taken: 0.016794681549072266, Downloaded 409 bytes
Time taken: 0.014864921569824219, Downloaded 409 bytes
Time taken: 0.015237808227539062, Downloaded 409 bytes
Time taken: 0.018918752670288086, Downloaded 409 bytes
Time taken: 0.016691207885742188, Downloaded 409 bytes
Time taken: 0.012588977813720703, Downloaded 409 bytes
Time taken: 0.019951581954956055, Downloaded 409 bytes
Time taken: 0.01525735855102539, Downloaded 409 bytes
Time taken: 0.

<coroutine object ClientSession.close at 0x000001C1A4EDE740>

In [119]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

import aiohttp
import numpy as np
import pandas as pd


def _new_session(endpoint, timeout: int = None):
    auth = None
    if endpoint.username:
        auth = aiohttp.BasicAuth(endpoint.username, endpoint.password)
    timeout = aiohttp.ClientTimeout(total=timeout) if timeout else aiohttp.ClientTimeout(total=300)
    return aiohttp.ClientSession(
        auth=auth,
        # read_bufsize=4 * 1024 * 1024,
        timeout=timeout)

def _auth_headers(endpoint) -> dict[str, str]:
    if endpoint.token:
        return {'Authorization': f'Bearer {endpoint.token}'}
    return {}

async def query_questdb_with_session(session, endpoint):
    """Query QuestDB using an existing session"""
    start_time = time.time()
    query = "SELECT * FROM option_data ORDER BY timestamp DESC LIMIT 1"
    
    url = f'{endpoint.url}/exp'
    # Removed redundant REST API limit since SQL already limits to 1 row
    params = [('query', query)]
    
    async with session.get(url=url, params=params, headers=_auth_headers(endpoint)) as resp:
        if resp.status != 200:
            raise ValueError(f'Failed to query QuestDB: {resp.status} {resp.reason}')
        
        buf = await resp.content.read()
        end_time = time.time()
        print(f"Time taken: {end_time-start_time:.4f}s, Downloaded {len(buf)} bytes")
        return buf

async def query_questdb(endpoint):
    """Single query with its own session (for standalone use)"""
    async with _new_session(endpoint) as session:
        return await query_questdb_with_session(session, endpoint)

async def run_multiple_queries_shared_session(endpoint, num_queries=100):
    """Run multiple queries with a shared session - more efficient"""
    async with _new_session(endpoint) as session:
        tasks = [query_questdb_with_session(session, endpoint) for _ in range(num_queries)]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Count successful vs failed requests
        successful = sum(1 for r in results if not isinstance(r, Exception))
        failed = len(results) - successful
        print(f"Completed {successful} successful queries, {failed} failed")
        return results

async def run_multiple_queries_separate_sessions(endpoint, num_queries=100):
    """Run multiple queries with separate sessions - less efficient but more isolated"""
    tasks = [query_questdb(endpoint) for _ in range(num_queries)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Count successful vs failed requests
    successful = sum(1 for r in results if not isinstance(r, Exception))
    failed = len(results) - successful
    print(f"Completed {successful} successful queries, {failed} failed")
    return results

async def main():
    # Configure your QuestDB endpoint
    endpoint = Endpoint(host='qdb3.twocc.in', port=None, https=True, username='2Cents', password='2Cents1012cc')

    
    print("Running single query...")
    await query_questdb(endpoint)
    
    print("\nRunning 100 concurrent queries with shared session...")
    total_start = time.time()
    await run_multiple_queries_shared_session(endpoint, 100)
    total_end = time.time()
    print(f"Total time for 100 queries (shared session): {total_end-total_start:.4f}s")
    
    print("\nRunning 100 concurrent queries with separate sessions...")
    total_start = time.time()
    await run_multiple_queries_separate_sessions(endpoint, 100)
    total_end = time.time()
    print(f"Total time for 100 queries (separate sessions): {total_end-total_start:.4f}s")

# Proper way to run async code
if __name__ == "__main__":
    asyncio.run(main())

Running single query...
Time taken: 0.0896s, Downloaded 409 bytes

Running 100 concurrent queries with shared session...
Time taken: 0.1843s, Downloaded 409 bytes
Time taken: 0.1833s, Downloaded 409 bytes
Time taken: 0.1843s, Downloaded 409 bytes
Time taken: 0.1875s, Downloaded 409 bytes
Time taken: 0.2081s, Downloaded 409 bytes
Time taken: 0.2071s, Downloaded 409 bytes
Time taken: 0.2081s, Downloaded 409 bytes
Time taken: 0.2071s, Downloaded 409 bytes
Time taken: 0.2091s, Downloaded 409 bytes
Time taken: 0.2061s, Downloaded 409 bytes
Time taken: 0.2061s, Downloaded 409 bytes
Time taken: 0.2061s, Downloaded 409 bytes
Time taken: 0.2071s, Downloaded 409 bytes
Time taken: 0.2071s, Downloaded 409 bytes
Time taken: 0.2050s, Downloaded 409 bytes
Time taken: 0.2081s, Downloaded 409 bytes
Time taken: 0.2167s, Downloaded 409 bytes
Time taken: 0.2167s, Downloaded 409 bytes
Time taken: 0.2137s, Downloaded 409 bytes
Time taken: 0.2126s, Downloaded 409 bytes
Time taken: 0.2147s, Downloaded 409 byt